# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import idx2numpy
import imageio
from idx_tools import Idx

%matplotlib inline

In [ ]:
## This is the part for extracting digits from laser images

### Part 1 - character images extraction, clustering and converting to Idx format

In [ ]:
# Renaming the files 
import os
os.getcwd()
collection = "./Chip_photos"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("./Chip_photos/" + filename, "./Chip_photos/" + "Image_"+ str(i) + ".jpg")

In [ ]:
import cv2  # Computer vision library
import imageio
import os
import numpy as np

# Read the color image

for image in os.listdir('./Chip_photos/'):
    image = cv2.imread(os.path.join('./Chip_photos/', image))
    
    #new_image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 13, 2)
    inverted_binary = ~binary
    contours, hierarchy = cv2.findContours(inverted_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    with_contours = cv2.drawContours(gray, contours, 3,(0,0,255),1)
    
    
    path='./char_7'
    n = len(os.listdir('./char_7')) + 1
    n_roi = np.shape(contours)[0]
   
    for c in contours:
##for c in range(n_roi):
        x,y,w,h = cv2.boundingRect(c)
        if (cv2.contourArea(c)) > 50:
        #cv2.rectangle(with_contours,(x,y), (x+w,y+h), (255,0,0), 1)
            cv2.rectangle(image,(x,y), (x+w,y+h), (255,0,0), 1)
            ROI = with_contours[y:y+h, x:x+w]
        #cv2.imwrite('./char_3/ROI_{}.jpg'.format(ROI_number), ROI)
        #imageio.imwrite('{}/char{}.jpg'.format(ROI_number), ROI)
            imageio.imwrite('{}/char{}.jpg'.format(path, n), ROI)
        #ROI_number += 1
            n += 1
    
    

In [ ]:
## This is the part for clustering and converting images into idx format for NN 

In [ ]:
#Converting some of the images to gray scale to be consistant
import cv2
import imageio
import os
import numpy as np
import glob


for image in os.listdir('./color_characters/'):
    image = cv2.imread(os.path.join('./color_characters/', image))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    path='./characters_gray'
    n = len(os.listdir('./characters_gray')) + 1
    imageio.imwrite('{}/char{}.jpg.jpg'.format(path, n), gray)


In [ ]:
# Renaming the files after converting the rest to thr gray scale
import os
os.getcwd()
collection = "./all_characters"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("./all_characters/" + filename, "./all_characters/" + "char_"+ str(i) + ".jpg")

In [ ]:
# Resizing, renaming, clustering 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
# Path to the individual letters
data_path = './all_characters/'

# Target image size
convSize = [28, 28]

# File names of all sample images
files = [data_path + x for x in os.listdir(data_path)]

# Resize images and append to a numpy array
images = []
for file in files:
    img = cv2.imread(file)[:, :, 0]
    img = cv2.resize(img, (convSize[0], convSize[1]))
    img = img.reshape(convSize[0] * convSize[1])
    images.append(img)
    
images = np.array(images, dtype='float64')

In [ ]:
img.shape

In [ ]:
# Apply StandardScaler on the letter data
scaler = StandardScaler()
scaler.fit(images)
scaled = scaler.transform(images)

In [ ]:
# Calculate the first 25 principal components
pca = PCA(n_components=25)
pca.fit(scaled)
pca_img = pca.transform(scaled)

In [ ]:
#Use K-Means clustering to group the data into 100 clusters
nClusters = 100
kmeans = KMeans(n_clusters=nClusters, random_state=0).fit(pca_img)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')


ax.scatter(pca_img[:, 0], pca_img[:, 1], pca_img[:, 2], c=kmeans.labels_)

ax.set_xlabel('component 1', fontsize=18)
ax.set_ylabel('component 2', fontsize=18)
ax.set_zlabel('component 3', fontsize=18)

plt.show()

In [ ]:
path = './clustered'

if not os.path.isdir(path):
    os.mkdir(path)

n = 0
for i in range(kmeans.labels_.max()):
    
    cluster_path = '{}/{}'.format(path, i)
    
    if not os.path.isdir(cluster_path):
        os.mkdir(cluster_path)
    
    tmp = images[kmeans.labels_ == kmeans.labels_[i]]
    
    for j in range(np.shape(tmp)[0]):
        tmpImg = np.reshape(tmp[j], convSize).astype(np.uint8)
        imageio.imwrite('{}/{}.jpg'.format(cluster_path, n), tmpImg)
        n += 1
        
# Delete the un-clustered data
[os.remove(data_path + x) for x in os.listdir(data_path)]
os.rmdir(data_path)

In [ ]:
# Where is the sorted data located?
dataset_path = './data/' 

# Where should the converted data be stored?
dest_folder = './datasetTMP/' 

# What is the ratio of train vs. test data?
train_percent = 0.7

# Subfolders for structuring the sorted data
train_path = dataset_path + 'train/'
test_path = dataset_path + 'test/'

# Find all folders in the sorted data
folders = os.listdir(dataset_path)

# Create folders
if not os.path.isdir(train_path):
    os.mkdir(train_path)
    os.mkdir(test_path)
    
if not os.path.isdir(dest_folder):
    os.mkdir(dest_folder)
    
# Go through all folders in the sorted data and split into train and test    
for char in folders:
    char_path = dataset_path + char + '/'
    train_folder = train_path + char + '/'
    test_folder = test_path + char + '/'
    
    samples = os.listdir(char_path)
    n_samples = len(samples)
    n_train = round(train_percent * n_samples)
    
    sel = np.arange(n_samples)
    np.random.shuffle(sel)
    
    idx_train = sel[0:n_train] 
    idx_test = sel[n_train:]
    
    if not os.path.isdir(train_folder):
        os.mkdir(train_folder)
        
    if not os.path.isdir(test_folder):
        os.mkdir(test_folder)
    
    [os.rename(char_path + samples[x], train_folder + samples[x]) for x in idx_train]
    [os.rename(char_path + samples[x], test_folder + samples[x]) for x in idx_test]
    
    os.rmdir(char_path)
    
# Convert data to idx format    
Idx.save_idx(train_path)
Idx.save_idx(test_path)

# Move converted dataset to target folder
os.rename(train_path + 'images.idx3-ubyte', dest_folder + 'train-images.idx3-ubyte')
os.rename(train_path + 'labels.idx3-ubyte', dest_folder + 'train-labels.idx3-ubyte')
os.rename(test_path + 'images.idx3-ubyte', dest_folder + 'test-images.idx3-ubyte')
os.rename(test_path + 'labels.idx3-ubyte', dest_folder + 'test-labels.idx3-ubyte')

In [ ]:
chars = os.listdir('./data/train')
dest_folder = './datasetTMP/' 

labelFile = open('{}/labels.txt'.format(dest_folder), "w")
for char in chars:
    
    #if char.endswith('_'):
        #char = char[:-1].upper()
    
    # write line to output file
    labelFile.write(char)
    labelFile.write("\n")
labelFile.close()

### Part 2- training neural network 

In [ ]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

import os

from time import strftime
from PIL import Image 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard

from sklearn.metrics import confusion_matrix
from IPython.display import display
from keras.preprocessing.image import array_to_img

from idx_tools import Idx
import cv2
import random

%matplotlib inline

In [ ]:
# Read the data 
filename = './dataset/train-images.idx3-ubyte'
train_images = Idx.load_idx(filename)

# Read the labels
filename = './dataset/train-labels.idx3-ubyte'
train_labels = Idx.load_labels(filename)

# Read the data 
filename = './dataset/test-images.idx3-ubyte'
test_images = Idx.load_idx(filename)

# Read the labels
filename = './dataset/test-labels.idx3-ubyte'
test_labels = Idx.load_labels(filename)

# Read the letter names for each label
filename = './dataset/labels.txt'

file = open(filename,"r") 
class_names =  [str.split(x) for x in file.readlines()]

In [ ]:
# Plot some random examples
n_images = train_images.shape[0]
n_plots = 10

fig, ax = plt.subplots(1, n_plots, figsize=(18, 18))
for i in range(n_plots):
    image_num = np.random.randint(low=0, high=n_images)
    ax[i].imshow(train_images[image_num], cmap='Greys')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title(class_names[train_labels[image_num]][0])

In [ ]:
# Normalize the data
train_images = train_images / 255.0
test_images = test_images / 255.0

# How many categories do we have in the dataset
n_categories = len(class_names)


In [ ]:
# What are the dimensions of each image
x_size = train_images.shape[1]
y_size = train_images.shape[2]

In [ ]:
random.shuffle(train_images)

In [ ]:
random.shuffle(test_images)

In [ ]:
batch_size = 128
epochs = 50

img_rows = train_images.shape[1]
img_cols = train_images.shape[2]

n_categories = len(class_names)

train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)

#Creating validation set

#validation_size = 10000

#x_val = train_images[:validation_size]
#y_val = train_labels[:validation_size]

#train_images = train_images[validation_size:]
#train_labels = train_labels[validation_size:]

input_shape = (img_rows, img_cols, 1)

model_3 = Sequential()
model_3.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model_3.add(Conv2D(64, (5, 5), activation='relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.5))
model_3.add(Flatten())
model_3.add(Dense(128, activation='relu'))
model_3.add(Dropout(0.3))
model_3.add(Dense(80))
model_3.add(Dense(n_categories, activation='softmax'))

model_3.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.00001),
              metrics=['accuracy'])

history_3 =  model_3.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, test_labels))

score = model_3.evaluate(test_images, test_labels, verbose=0)
print(score)

In [ ]:
# Set up the plots
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot the data
ax[0].plot(history_3.history['loss'])
ax[0].set_title('Loss over epochs', fontsize=23)
ax[0].set_xlabel('epoch', fontsize=16)
ax[0].set_ylabel('loss', fontsize=16)

ax[1].plot(history_3.history['accuracy'])
ax[1].set_title('Accuracy over epochs', fontsize=23)
ax[1].set_xlabel('epoch', fontsize=16)
ax[1].set_ylabel('accuracy', fontsize=16)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np




   
def plot_image(i, predictions_array, true_label, img, category_names):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(category_names[predicted_label],
                                         100 * np.max(predictions_array),
                                         category_names[true_label]),
               color=color)


def plot_value_array(i, predictions_array, true_label, n_categories):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    this_plot = plt.bar(range(n_categories), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    this_plot[predicted_label].set_color('red')
    this_plot[true_label].set_color('blue')

In [ ]:
train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)

predictions = model_3.predict(test_images)

# Read the data 
filename = './dataset/test-images.idx3-ubyte'
test_images = Idx.load_idx(filename)

# Read the labels
filename = './dataset/test-labels.idx3-ubyte'
test_labels = Idx.load_labels(filename)


# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 15
num_cols = 15
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    rand_image = np.random.randint(low=0, high=test_images.shape[0])
    
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    
    plot_image(rand_image, predictions, test_labels, test_images, class_names)
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
    plot_value_array(rand_image, predictions, test_labels, n_categories)
plt.show()

In [ ]:
img = cv2.imread('test_1.jpg', cv2.IMREAD_GRAYSCALE)
#test_0 = test_0/255.0
#test_0 = cv2.resize(test_0, (28,28))
img = np.array(img)
img = 255 - img
img = img / 255

In [ ]:
result_1 = model_3.predict_classes(img.reshape(-1, 28, 28, 1), verbose=1)

In [ ]:
print(class_names[result_1[0]])

In [ ]:
if not os.path.isdir('./model_3'):
    os.mkdir('./model_3')

# Save the model structure to JSON file
model_3_json = model_3.to_json()
with open("./model_3/model_3.json", "w") as json_file:
    json_file.write(model_3_json)

# Save weights to HDF5 file
model_3.save_weights("./model_3/model_3.h5")
print("Model_3 saved")

### Part 3 - testing the model on real life images 

In [ ]:
# import the necessary packages
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import numpy as np
import argparse
import imutils
import cv2
from keras.models import model_from_json

print("[INFO] loading handwriting OCR model...")
model_path = "./model_3/model_3.json"
weights_path = "./model_3/model_3.h5"
# Load the model from file
model_conv_file = open(model_path, 'r')
model_conv = model_conv_file.read()
model_conv = model_from_json(model_conv)
model_conv_file.close()
# Load the weights from file and add them to the model
model_conv.load_weights(weights_path)
print("ConvNet model and weights loaded")
# Compile the model
model_conv.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#%% 
filename = './dataset/labels.txt'
file = open(filename,"r") 
class_names =  [str.split(x) for x in file.readlines()]

# reading image
#image = cv2.imread('images\image_2762.jpg')
image = cv2.imread('./Images_5/image_53.jpg')

dim = image.shape
w1=3*dim[1]/100
w2=dim[1]/2
h1=3*dim[0]/100
h2=dim[0]/2



gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
# initialize the list of contour bounding boxes and associated
# characters that we'll be OCR'ing
chars = []
#display
#cv2.imshow('contours',edged)
#cv2.waitKey(0)
#%%
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    if (w >= w1 and w <= w2) and (h >= h1 and h <= h2):
    # extract the character and threshold it to make the character
    # appear as *white* (foreground) on a *black* background, then
   # grab the width and height of the thresholded image
       roi = gray[y:y + h, x:x + w]
       thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        (tH, tW) = thresh.shape
    # if the width is greater than the height, resize along the
    # width dimension
    if tW > tH:
        thresh = imutils.resize(thresh, width=28)
    # otherwise, resize along the height
    else:
        thresh = imutils.resize(thresh, height=28)    
# re-grab the image dimensions (now that its been resized)
# and then determine how much we need to pad the width and
# height such that our image will be 32x32
        (tH, tW) = thresh.shape
        dX = int(max(0, 28 - tW) / 2.0)
        dY = int(max(0, 28 - tH) / 2.0)
# pad the image and force 32x32 dimensions
        padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY, left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,value=(0, 0, 0))
        padded = cv2.resize(padded, (28, 28))
    # prepare the padded image for classification via our
     # handwriting OCR model
        padded = padded.astype("float32") / 255.0
        padded = np.expand_dims(padded, axis=-1)
     # update our list of characters that will be OCR'd
        chars.append((padded, (x, y, w, h)))
            
            # re-grab the image dimensions (now that its been resized)
    # and then determine how much we need to pad the width and
    # height such that our image will be 28x28
(tH, tW) = thresh.shape
dX = int(max(0, 28 - tW) / 2.0)
dY = int(max(0, 28 - tH) / 2.0)
# pad the image and force 32x32 dimensions
padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,value=(0, 0, 0))
padded = cv2.resize(padded, (28, 28))
# prepare the padded image for classification via our
# handwriting OCR model
padded = padded.astype("float32") / 255.0
padded = np.expand_dims(padded, axis=-1)
# update our list of characters that will be OCR'd
chars.append((padded, (x, y, w, h)))
            
# extract the bounding box locations and padded characters
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")
# OCR the characters using our handwriting recognition model
preds = model_conv.predict(chars)
# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHJKLMN"
labelNames = [l for l in labelNames]


#%%
# loop over the predictions and bounding box locations together
results=[]
count = 0
r=0
for (pred, (x, y, w, h)) in zip(preds, boxes):
    r=h/w
    if r>=1.3 and r<=3:
 # find the index of the label with the largest corresponding  
 # probability, then extract the probability and label
        i = np.argmax(pred)
        prob = pred[i]
        label = labelNames[i]
        print("[INFO] {} - {:.2f}% - {:.2f} - x:{:.2f} - y:{:.2f}".format(label, prob * 100, (h/w), x, y))
        
        results.append([label,prob,(h/w),x,y,(x+y)])
        count = count+1
    
#if prob >= 0.99:
 # draw the prediction on the image
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
       

 #now get best guess by taking 4 highest %
results.sort(key=lambda results:results[1])
results.reverse()
answers = results[0:8]
answers.sort(key=lambda answers:answers[2])

final_answers =[]
for l in answers:
    if l not in final_answers:
        final_answers.append(l)
final_answers.sort(key=lambda final_answers:final_answers[5])
just_names = [item[0] for item in final_answers]
print('Final Answer: '+''.join(just_names[0:2])+'_'+''.join(just_names[2:4]))


# show the image
cv2.imshow("Image", image)
cv2.waitKey(0)
